In [0]:
catalog_name = dbutils.widgets.text("catalog_name", "")
catalog_name = dbutils.widgets.get("catalog_name")
bronze_schema = dbutils.widgets.text("bronze_schema", "")
bronze_schema = dbutils.widgets.get("bronze_schema")
silver_schema = dbutils.widgets.text("silver_schema", "")
silver_schema = dbutils.widgets.get("silver_schema")
input_table_product = dbutils.widgets.text("input_table_product", "")
input_table_product = dbutils.widgets.get("input_table_product")
output_table_product = dbutils.widgets.text("output_table_product", "")
output_table_product = dbutils.widgets.get("output_table_product")

In [0]:

from pyspark.sql.window import Window
from pyspark.sql import functions as F
from pyspark.sql.functions import col, trim, initcap, when, lit, year, month, dayofmonth, concat, concat_ws
from delta.tables import DeltaTable
from pyspark.sql.utils import AnalysisException
import time

In [0]:
schema_auditoria ="auditoria"

In [0]:
sql_catalog = f"""
    CREATE CATALOG IF NOT EXISTS {catalog_name}
"""

spark.sql(sql_catalog)

In [0]:
sql_silver = f"""
    CREATE SCHEMA IF NOT EXISTS {catalog_name}.{silver_schema}
"""

spark.sql(sql_silver)

In [0]:
silver_productos = (
    spark.table(f"{catalog_name}.{bronze_schema}.{input_table_product}")
    .withColumn("id_producto", F.col("id_producto").cast("bigint"))
    .withColumn("nombre", initcap(F.trim(F.col("nombre"))))
    .withColumn("categoria", initcap(F.trim(F.col("categoria"))))
    .withColumn("precio", F.col("precio").cast("decimal(18,2)"))
    .withColumn("updated_at", F.current_timestamp())
    .dropna(subset=["id_producto"])
    .dropDuplicates(["id_producto"])
)

In [0]:
display(silver_productos)

In [0]:
spark.sql(f"""
CREATE TABLE IF NOT EXISTS {catalog_name}.{silver_schema}.{input_table_product} (
  id_producto BIGINT,
  nombre STRING,
  categoria STRING,
  precio DECIMAL(18,2),
  updated_at TIMESTAMP
) USING DELTA
""")

In [0]:

delta_target = DeltaTable.forName(spark, f"{catalog_name}.{silver_schema}.{output_table_product}")

merge_start = time.time()
delta_target.alias("tgt") \
  .merge(
    source=silver_productos.alias("src"),
    condition="tgt.id_producto = src.id_producto"
  ) \
  .whenMatchedUpdateAll(
    condition="src.updated_at > tgt.updated_at"
  ) \
  .whenNotMatchedInsertAll() \
  .execute()
merge_end = time.time()

In [0]:
spark.table(f"{catalog_name}.{silver_schema}.{output_table_product}").show(10)

In [0]:
import time
job_id = int(time.time())           # ID único basado en timestamp
job_run_id = int(time.time() * 100) # más granular
task_run_id = 1                     # o secuencial si prefieres


In [0]:
rows_in = silver_productos.count()
start = time.time()
spark.sql(f"""
INSERT INTO {catalog_name}.{schema_auditoria}.ingestion_log VALUES (
  {job_id},{job_run_id},{task_run_id},
  current_timestamp(), current_timestamp(),
  {int(time.time()-start)},
  'SUCCESS','productos','silver',
  {rows_in},{rows_in},0,0,0,{int(merge_end-merge_start)},current_timestamp()
)
""")

In [0]:
display(spark.table(f"{catalog_name}.{schema_auditoria}.ingestion_log"))
